# Source Code Analysis

## Initialization

In [1]:
import os
import sys

def add_path(path):
    if path not in sys.path:
        sys.path.insert(0, path)
        sys.path.append(path)
add_path('/home/jjian03/anaconda3/lib/python3.7/site-packages')
add_path('/home/jjian03/iconference_followup_study')


### Load Data

In [2]:
import time
import datetime
import pandas as pd

seed = 77

data_file = '../data/untrunc_data_cleaned_url.csv'

raw_data = pd.read_table(data_file, sep=',', index_col=0)
raw_data = raw_data.dropna()

raw_data.info()

print(f'raw_data: {raw_data.shape}')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58871 entries, 0 to 58909
Data columns (total 42 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   protocol_type                           58871 non-null  float64
 1   has_www                                 58871 non-null  float64
 2   has_iframe                              58871 non-null  float64
 3   int                                     58871 non-null  float64
 4   org                                     58871 non-null  float64
 5   gov                                     58871 non-null  float64
 6   in                                      58871 non-null  float64
 7   eu                                      58871 non-null  float64
 8   cn                                      58871 non-null  float64
 9   kr                                      58871 non-null  float64
 10  url_depth                               58871 non-null  fl

### Train Test Split

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


x = raw_data.drop(['label', 'first_appear', 'url'], axis=1)
y = raw_data.label
y = preprocessing.StandardScaler().fit_transform(y.values.reshape(-1, 1))
y = pd.DataFrame(y).iloc[:,0]


X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size = 0.33, random_state=seed)

In [4]:
import gc
import multiprocessing

import warnings
warnings.filterwarnings("ignore")


cpu_cnt = multiprocessing.cpu_count()
allocated_cpu = cpu_cnt
print(f"Allocated {allocated_cpu} CPUs")
gc.collect()

Allocated 16 CPUs


20

In [5]:
import numpy as np

from sklearn import preprocessing, clone
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils.validation import _check_fit_params


class VerboseGridSearchCV(GridSearchCV):

    def __init__(self, estimator, param_grid, *, scoring=None,
                 n_jobs=None, iid='deprecated', refit=True, cv=None,
                 verbose=0, pre_dispatch='2*n_jobs',
                 error_score=np.nan, return_train_score=False):
        super().__init__(
            estimator=estimator, param_grid=param_grid, scoring=scoring,
            n_jobs=n_jobs, iid=iid, refit=refit, cv=cv, verbose=verbose,
            pre_dispatch=pre_dispatch, error_score=error_score,
            return_train_score=return_train_score)
        self.estimators_ = list()
        self.params_ = list()

    def fit(self, X, y=None, *, groups=None, **fit_params):
        super(VerboseGridSearchCV, self).fit(X, y=y, groups=groups, **fit_params)
        if not self.refit:
            return self
        results = self.cv_results_

        fit_params = _check_fit_params(X, fit_params)
        for idx in range(len(results["params"])):
            params = results["params"][idx]
            estimator = clone(clone(clone(self.estimator)).set_params(**params))
            if y is not None:
                estimator.fit(X, y, **fit_params)
            else:
                estimator.fit(X, **fit_params)

            self.estimators_.append(estimator)
            self.params_.append(params)

        grid.params_ = pd.DataFrame.from_dict(grid.params_)
        return self

#### L-BFGS-B

In [6]:
param_rf = {
    'max_depth': [*np.arange(1,35,2)],
    'n_estimators': [*np.arange(120,200,50)],

    'min_samples_split': [*np.arange(10,50,5)],
    'min_samples_leaf': [*np.arange(5,40,3)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',
    
    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


exception calling callback for <Future at 0x7f0e561e3d30 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/jjian03/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/jjian03/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 347, in __call__
    self.parallel.dispatch_next()
  File "/home/jjian03/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 780, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/jjian03/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/jjian03/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "/home/jjian03/anaconda3/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 529, in apply_async

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {EXIT(2), EXIT(2)}

In [ ]:
feature_importance = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.feature_importances_) \
    .apply(pd.Series)
feature_importance.columns = X_train.columns
feature_importance = grid.params_.join(feature_importance)

feature_importance


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


out_of_bag_score = pd.Series(grid.estimators_).apply(lambda estimator: estimator.oob_score_)
out_of_bag_score.name = 'out_of_bag_score'

plt.clf()
plt.figure(figsize=(20, 5))

sns.boxplot(x=out_of_bag_score)

plt.title('Distribution of MSE(Out of Bag Score)', fontSize=25)


In [ ]:
def get_pseudo_r2(y_true, y_hat):
    correlation_matrix = np.corrcoef(
        y_true.astype(float), 
        y_hat.astype(float)
    )
    correlation_xy = correlation_matrix[0,1]
    return correlation_xy**2


r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
performance

In [ ]:
plt.clf()

f, ax = plt.subplots(2, 2, figsize=(15,20))
sns.lineplot(y="r_square", x="max_depth", hue='data_type', style='data_type', data=performance, ax=ax[0, 0])
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0, 1])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1, 0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1, 1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'n_estimators': [*np.arange(200,400,50)],

    'min_samples_split': [*np.arange(20,90,5)],
    'min_samples_leaf': [*np.arange(40,240,3)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    max_depth=20,
    
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(3, 1, figsize=(17,20))
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[2])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'n_estimators': [*np.arange(200,2000,100)],
    'min_samples_split': [*np.arange(150,400,20)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    max_depth=10,
    min_samples_leaf=120,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(17,10))
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'n_estimators': [*np.arange(200,3000,50)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    max_depth=10,
    min_samples_leaf=120,
    min_samples_split=250,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

plt.figure(figsize=(20, 5))
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance)

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'max_depth': [*np.arange(7,15,1)],
    'min_samples_leaf': [*np.arange(100,150,10)],
    'min_samples_split': [*np.arange(200,300,10)],
    'n_estimators': [*np.arange(1400,1700,50)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 2, figsize=(15,15))
sns.lineplot(y="r_square", x="max_depth", hue='data_type', style='data_type', data=performance, ax=ax[0, 0])
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0, 1])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1, 0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1, 1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(100,4000,200)],
    'min_samples_split': [*np.arange(100,4000,200)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    n_estimators=50,
    max_depth=8,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(250,1000,50)],
    'min_samples_split': [*np.arange(100,8000,200)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    n_estimators=50,
    max_depth=8,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(250,2500,50)],
    'min_samples_split': [*np.arange(100,5000,100)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    n_estimators=50,
    max_depth=8,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(100,1250,10)],
    'min_samples_split': [*np.arange(100,1000,5)],
}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,

    n_estimators=50,
    max_depth=8,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff

In [ ]:
plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_leaf", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square_diff", x="min_samples_split", data=performance_diff, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)

In [ ]:
param_rf = {
    'min_samples_leaf': [220],
    'min_samples_split': [300],
    'n_estimators': [1500],
    'max_depth': [8],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
y_hat_train = grid.best_estimator_.predict(X_train)
print(f'R Square on Training set: {get_pseudo_r2(y_train, y_hat_train)}')
y_hat_test = grid.best_estimator_.predict(X_test)
print(f'R Square on Testing set:  {get_pseudo_r2(y_test, y_hat_test)}')

In [ ]:
import matplotlib


def plot_feature_importance(reg_coef, col_names, title):
    reg_coef = pd.Series(reg_coef, index=col_names)
    reg_coef = reg_coef.sort_values()
    matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
    reg_coef.plot(kind="barh",)
    plt.title(title, fontsize=15)


In [ ]:
feature_names = [*X_train.columns]

# Get the model without any penalty term
coef = grid.best_estimator_.feature_importances_
plot_feature_importance(
    coef, feature_names, 
    "Coefficients of the Random Forest")

In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(2,10,1)],
    'min_samples_split': [*np.arange(2,20,1)],
    'n_estimators': [1500],
    'max_depth': [8],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_leaf", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square_diff", x="min_samples_split", data=performance_diff, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)

In [ ]:
param_rf = {
    'min_samples_leaf': [220],
    'min_samples_split': [300],
    'n_estimators': [*np.arange(10,1500,5)],
    'max_depth': [8],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="n_estimators", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [220],
    'min_samples_split': [300],
    'n_estimators': [*np.arange(2,300,1)],
    'max_depth': [8],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="n_estimators", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [220],
    'min_samples_split': [300],
    'n_estimators': [*np.arange(10,20,1)],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="n_estimators", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(150,250,1)],
    'min_samples_split': [300],
    'n_estimators': [14],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_leaf", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(220,250,1)],
    'min_samples_split': [300],
    'n_estimators': [14],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_leaf", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [228],
    'min_samples_split': [*np.arange(2,1000,1)],
    'n_estimators': [14],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_split", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [230],
    'min_samples_split': [*np.arange(475,500,1)],
    'n_estimators': [14],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_split", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [230],
    'min_samples_split': [490],
    'n_estimators': [*np.arange(20,60,1)],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="n_estimators", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="n_estimators", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [*np.arange(230,240,1)],
    'min_samples_split': [490],
    'n_estimators': [30],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_leaf", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_leaf", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [232],
    'min_samples_split': [*np.arange(480,500,1)],
    'n_estimators': [30],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
r2_train = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_train)) \
    .apply(lambda y_hat: get_pseudo_r2(y_train, y_hat))
r2_test = pd.Series(grid.estimators_) \
    .apply(lambda estimator: estimator.predict(X_test)) \
    .apply(lambda y_hat: get_pseudo_r2(y_test, y_hat))

r2_train.name = 'r_square'
r2_test.name = 'r_square'

performance_train = grid.params_.join(r2_train)
performance_test = grid.params_.join(r2_test)
performance_train.loc[:, 'data_type'] = 'Training Set'
performance_test.loc[:, 'data_type'] = 'Testing Set'

performance = performance_train.append(performance_test)

r2_diff = r2_train-r2_test
r2_diff.name = 'r_square_diff'

performance_diff = grid.params_.join(r2_diff)

performance_diff


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()


plt.clf()

f, ax = plt.subplots(2, 1, figsize=(15,10))
sns.lineplot(y="r_square_diff", x="min_samples_split", data=performance_diff, ax=ax[0])
sns.lineplot(y="r_square", x="min_samples_split", hue='data_type', style='data_type', data=performance, ax=ax[1])

plt.suptitle("Generalization ability inspection", size=26)


In [ ]:
param_rf = {
    'min_samples_leaf': [232],
    'min_samples_split': [490],
    'n_estimators': [30],
    'max_depth': [11],

}

rf = RandomForestRegressor(
    criterion="mse",
    verbose=False,
    bootstrap=True,
    random_state=seed,
    warm_start=True,
    oob_score = True,
    min_weight_fraction_leaf = 0.,
    max_features='sqrt',

    max_leaf_nodes=None,
    min_impurity_decrease=0.,
    max_samples=None,
    ccp_alpha=0.0,
)

grid = VerboseGridSearchCV(
    n_jobs=allocated_cpu,
    cv=5,
    estimator=rf,
    param_grid=param_rf,
)
grid.fit(X_train, y_train)


In [ ]:
y_hat_train = grid.best_estimator_.predict(X_train)
print(f'R Square on Training set: {get_pseudo_r2(y_train, y_hat_train)}')
y_hat_test = grid.best_estimator_.predict(X_test)
print(f'R Square on Testing set:  {get_pseudo_r2(y_test, y_hat_test)}')

In [ ]:
feature_names = [*X_train.columns]

# Get the model without any penalty term
coef = grid.best_estimator_.feature_importances_
plot_feature_importance(
    coef, feature_names, 
    "Coefficients of the Random Forest")

In [ ]:
import shap


shap.initjs()

explainer = shap.TreeExplainer(grid.best_estimator_)
shap_values = explainer.shap_values(X_test)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], X_test.iloc[0,:])

In [ ]:
# shap.force_plot(explainer.expected_value, shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")